In [1]:
#pip install pandas_read_xml

### Apple Health Data Try:

In [2]:
#Reading in xml files, when I thought I might need apple data:

In [1]:
import pandas_read_xml as pdx
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import json
import re

In [4]:
df = pdx.read_xml("./data/apple_health_export_brian_mclain/export.xml")

FileNotFoundError: [Errno 2] No such file or directory: './data/apple_health_export_brian_mclain/export.xml'

In [ ]:
getsizeof(df)

In [ ]:
df['Record']

In [ ]:
import pandas as pd
import xml.etree.ElementTree
import datetime

path_to_exportxml = "./data/apple_health_export_brian_mclain/export.xml"

def iter_records(healthdata):
    healthdata_attr = healthdata.attrib
    for rec in healthdata.iterfind('.//Record'):
        rec_dict = healthdata_attr.copy()
        rec_dict.update(healthdata.attrib)
        for k, v in rec.attrib.items():
            if 'date' in k.lower():
                rec_dict[k] = datetime.datetime.strptime(v, '%Y-%m-%d %H:%M:%S %z')
            else:
                rec_dict[k] = v
        yield rec_dict

e = xml.etree.ElementTree.parse(path_to_exportxml).getroot()
df = pd.DataFrame(list(iter_records(e)))

df.head()

In [ ]:
df['type'].unique()

In [ ]:
run_walk_df = df[(df['type'] == 'HKQuantityTypeIdentifierDistanceWalkingRunning')]

In [ ]:
df.to_csv('./data/b_mclain_raw.csv')

In [ ]:
df['value'] = pd.to_numeric(df['value'])
df['value']

In [ ]:
run_walk_df['type'].value_counts()

In [ ]:
run_walk_df['value'] = pd.to_numeric(run_walk_df['value'])

In [ ]:
run_walk_df['value']

In [ ]:
run_walk_df[run_walk_df['value']>.75]

In [ ]:
run_walk_df['creationDate'].unique()

In [ ]:
run_df = df[(df['type'] == 'HKQuantityTypeIdentifierDistanceWalkingRunning') | (df['type'] == 'HKQuantityTypeIdentifierWalkingSpeed')| (df['type'] == 'HKQuantityTypeIdentifierAppleExerciseTime')]

In [ ]:
len(run_df)

In [ ]:
run_df['date']=pd.to_datetime(run_df['creationDate'], format='%Y-%m-%d')

In [ ]:
run_df['date'].iloc[0].day

In [ ]:
day = []
month = []
year = []

for i in range(len(run_df)):    
    day.append(run_df['date'].iloc[i].day)
    month.append(run_df['date'].iloc[i].month)
    year.append(run_df['date'].iloc[i].year)

run_df['day'] = day
run_df['month'] = month
run_df['year'] = year

In [ ]:
run_df.head()

In [ ]:
run_df.reset_index(inplace=True)

In [ ]:
date_object = []
for i in range(len(run_df)):    
    date_object.append(str(run_df.iloc[0]['month'])+' '+str(run_df.iloc[0]['day'])+' '+str(run_df.iloc[0]['year']))
run_df['date_object']= date_object

In [ ]:
run_df.head()


### End Apple Health Kit Data

### Garmin Data Scratch Work:

In [ ]:
bkk = pd.read_csv('./data/raw/Garmin/ActivitiesBKK2.csv')

In [ ]:
bkk.head()

In [ ]:
bkk_running = bkk[bkk['Activity Type'] == 'Running']

In [ ]:
bkk_running['Distance'].describe()

In [ ]:
bkk_running['Distance'] = bkk_running['Distance'].astype('float')

In [ ]:
bkk_running['Distance'].describe()

In [ ]:
bkk_running[bkk_running['Distance'] >= 12.5]

In [ ]:
eb = pd.read_csv('./data/raw/Garmin/02_eric_bayless_updated.csv')

In [ ]:
eb['Race'].sum()

In [ ]:
eb[eb['Race'] != 0]

In [ ]:
jb = pd.read_csv('./data/raw/Garmin/06_jen-all-activities.csv')

In [ ]:
pd.to_numeric(jb['Distance'])

In [ ]:
#jb.drop(index=260, inplace=True)
jb[jb['Activity Type'] == 'Running']['Distance'].astype('float').max()

In [ ]:
jb[jb['Activity Type'] == 'Running']['Distance'].astype('float').describe()

In [ ]:
jb.loc[379]

In [ ]:
jb.head()

In [ ]:
lars = pd.read_csv('./data/raw/Lars-Strava/activities.csv')

In [ ]:
lars['Activity Description'].unique()

In [ ]:
joe = pd.read_csv('./data/raw/Garmin/09_Riggs Activities r4 w favorites.csv')

In [ ]:
#Joe Tagged his races and hard efforts as favorites
joe[joe['Favorite'] == True]

### Test importation of data and race dates and prep for simple time series model:

In [9]:
#ast and ast.literal_eval let us turn a string into a list pythonically https://stackoverflow.com/questions/1894269/how-to-convert-string-representation-of-list-to-a-list
import ast

In [10]:
n = 9
with open("./data/athletes.json") as json_file: 
    data = json.load(json_file)
    races = ast.literal_eval(data[f"{n}"]['race_date_list'])
races

['2016-04-12',
 '2016-07-17',
 '2016-09-25',
 '2016-09-25',
 '2016-12-03',
 '2016-12-17',
 '2017-01-07',
 '2017-01-28',
 '2017-04-09',
 '2017-04-09',
 '2017-04-22',
 '2017-04-22',
 '2017-04-22',
 '2017-11-11',
 '2017-12-02',
 '2017-12-16',
 '2018-03-24',
 '2018-04-21',
 '2019-11-16',
 '2020-03-05',
 '2020-05-10']

In [2]:
athlete_09_df = pd.read_csv('./data/raw/Garmin/09_Riggs Activities r4 w favorites.csv')

In [3]:
athlete_09_df['Avg Speed'].head()

0    22.0
1    7:20
2    21.8
3    8:08
4    21.6
Name: Avg Speed, dtype: object

In [4]:
print(re.findall("\d?\d:\d\d", athlete_09_df['Avg Speed'][1])[0])

7:20


In [27]:
def import_transf_garmin2(filepath, athlete_id, race_dates): #Filepath is the csv filepath, athlete_id is a counter variable, race_dates is the list of race dates
    
    #Read in the dataframe:
    df = pd.read_csv(filepath)
    #Found out this is very important in garmin files:
    df.replace('--', 0, inplace = True)
    
    #Create athlete_id, file type and race category column:
    df['athlete_id'] = athlete_id
    df['filetype'] = 'garmin'
    df['is_race'] = 0
    
    #Apparently some garmins save different columns:
    if 'Avg Speed' in df.columns:
        print("We have an Avg Speed Column!")
        df['Avg Pace'] = 0
        
        for i in range(len(df)):
            #print(df['Avg Speed'][i])
            try:
                df['Avg Pace'][i] = re.findall("\d?\d:\d\d", df['Avg Speed'][i])[0]
                #print(df['Avg_Pace'][i])
            except:
                df['Avg Pace'][i] = '00:00:00'
        #Best we can do for now is set the best pace to the average pace:
        df['Best Pace'] = df['Avg Pace']
    
    
    #Define the columns to have their data types transformed:
    obj_to_num_cols = ['Calories', 'Avg HR', 'Max HR', 'Aerobic TE', 'Avg Run Cadence', 'Max Run Cadence', 'Elev Gain', 'Elev Loss', 'Avg Run Cadence.1', 'Avg Power', 'Max Power']
    pace_cols = ['Avg Pace', 'Best Pace']
    
    #Convert the numeric columns:
    for col in obj_to_num_cols:
        try:
            df[col] = pd.to_numeric(df[col])
        except:
            try:
                df[col] = pd.to_numeric(df[col].str.extract(r'd+'))
            except:
                pass
    
    #Convert the pace columns:
    for col in pace_cols:
        for i in range(len(df[col])):
            try:
                df[col][i] = pd.to_timedelta('00:'+df[col][i])
            except:
                #print(f'col={col}')
                #print(f'i= {i}')
                #print(f'df[col][i] = {df[col][i]}')
                if df[col][i]==0:
                    df[col][i] = pd.to_timedelta('00:00:00')
                else:
                    pass
                pass
    
    #Clean up the column names:
    #This is to remove the (R) logo so it doesn't cause any issues later:
    r_logo = df.columns[df.columns.str.contains('Training Stress')][0][-1]
    #Clean up the column names:
    df.columns = df.columns.str.replace('/', '').str.lower().str.replace('.', '_').str.replace(' ', '_').str.replace('(', '').str.replace(')', '').str.replace(r_logo, '')
    df.set_index(df['date'], inplace = True)
    df.index = pd.to_datetime(df.index)
    ###Write a function to change the race dates in race_dates to a '1'
    
    #Write out the file to the data folder:
    df.to_csv(f'./data/clean/garmin_clean/clean_{athlete_id}.csv')

In [53]:
races = ['2020-10-05']

In [54]:
filepath = './data/raw/Garmin/06_jen-all-activities.csv'

In [55]:
import_transf_garmin2(filepath, 6, races)

In [29]:
clean_9 = pd.read_csv('./data/clean/garmin_clean/clean_9.csv')

In [31]:
clean_filepath = './data/clean/garmin_clean/clean_9.csv'

In [30]:
clean_9['is_race'].sum()

0

In [9]:
def import_transf_garmin(filepath, athlete_id, race_dates): #Filepath is the csv filepath, athlete_id is a counter variable, race_dates is the list of race dates
    
    #Read in the dataframe:
    df = pd.read_csv(filepath)
    #Found out this is very important in garmin files:
    df.replace('--', 0, inplace = True)
    
    #Create athlete_id, file type and race category column:
    df['athlete_id'] = athlete_id
    df['filetype'] = 'garmin'
    df['is_race'] = 0
    
    
    #Define the columns to have their data types transformed:
    obj_to_num_cols = ['Calories', 'Avg HR', 'Max HR', 'Aerobic TE', 'Avg Run Cadence', 'Max Run Cadence', 'Elev Gain', 'Elev Loss', 'Avg Run Cadence.1', 'Avg Power', 'Max Power']
    pace_cols = ['Avg Pace', 'Best Pace']
    
    #Convert the numeric columns:
    for col in obj_to_num_cols:
        try:
            df[col] = pd.to_numeric(df[col])
        except:
            try:
                df[col] = pd.to_numeric(df[col].str.extract(r'd+'))
            except:
                pass
    
    #Convert the pace columns:
    for col in pace_cols:
        for i in range(len(df[col])):
            try:
                df[col][i] = pd.to_timedelta('00:'+df[col][i])
            except:
                #print(f'col={col}')
                #print(f'i= {i}')
                #print(f'df[col][i] = {df[col][i]}')
                if df[col][i]==0:
                    df[col][i] = pd.to_timedelta('00:00:00')
                else:
                    pass
                pass
    
    #Clean up the column names:
    #This is to remove the (R) logo so it doesn't cause any issues later:
    r_logo = df.columns[df.columns.str.contains('Training Stress')][0][-1]
    #Clean up the column names:
    df.columns = df.columns.str.replace('/', '').str.lower().str.replace('.', '_').str.replace(' ', '_').str.replace('(', '').str.replace(')', '').str.replace(r_logo, '')
    df.set_index(df['date'], inplace = True)
    df.index = pd.to_datetime(df.index)
    ###Write a function to change the race dates in race_dates to a '1'
    
    #Write out the file to the data folder:
    df.to_csv(f'./data/clean/garmin_clean/clean_{athlete_id}.csv')

In [46]:
def set_up_for_prediction(df_filepath, athlete_dict):
    
    '''
    Inputs: csv filepath, and athlete dictionary
    -Dictionary contains an entry for race date, keyed to athlete id
    Outputs: 
        1) a csv with races indicated, indexed by time series
        2) a csv with summary values for 1 week, 2 weeks, 1 month, 2 months, 3 months
            - If values are not available, they will be 0s
    '''
    #Read in the csv:
    working = pd.read_csv(df_filepath, index_col='date').drop('date.1', axis =1)
    
    athlete_id = working['athlete_id'][0]
    
    #get the race dates for the selected athlete:
    race_dates = athlete_dict[athlete_id]['race_date_list']
    
    #Set the index as a datetime:
    working.index = pd.to_datetime(working.index)
    
    #Create a month interval: (we can multiply by 3 to get the 3 month values)
    month = timedelta(31)
    day = timedelta(1)
    
    #Create a time of day column:
    time_of_day = [re.findall("\d\d:\d\d:\d\d", working.index.to_series().astype('str')[i])[0] for i in range(len(working))]
    
    #Drop the hours and minutes and seconds:
    working.index = pd.DatetimeIndex(working.index).normalize()
    
    #Label all of the races:
    working['is_race'] = np.where(working.index.isin(race_dates), 1, 0)
    
    #Create a folder for the athlete to keep things tidy:
    parent_path = './data/races'
    directory = f'athlete_{athlete_id}'
    path = os.path.join(parent_path, directory)
    os.mkdir(path)
    
    
    #select the last month and last 3 months of data from the working file, and export them as separate csvs to the 'races' folder:
    
    for i in range(len(race_dates)):
        #i is the counter, find the 1 month and 3 month data:
        rd = pd.to_datetime(race_dates[i])
        race_1_mo_back = working[rd+day:rd-month]
        race_1_mo_back.sort_index(inplace=True)
        race_1_mo_back.to_csv(f'{path}/athlete_{athlete_id}_race_{i}_1_mo.csv')
        try:
            #This way if there is not 3 months data, it just skips out.
            race_3_mo_back = working[rd+day:(rd-month*3)]
            race_3_mo_back.sort_index(inplace=True)
            race_3_mo_back.to_csv(f'{path}/athlete_{athlete_id}_race_{i}_3_mo.csv')
        except:
            pass



In [37]:
test_athlete_dict = {2:{'race_date_list':['2020-05-09'],
                          'age':35, 'gender':'F', 
                          'race_count':1, 
                          'training_elevation':1000, 
                          'max_hr':175}, 
                     9:{'race_date_list':['2016-04-12', '2016-07-17', '2016-09-25', '2016-09-25', '2016-12-03', '2016-12-17', '2017-01-07', '2017-01-28', '2017-04-09', '2017-04-09', '2017-04-22', '2017-04-22', '2017-04-22', '2017-11-11', '2017-12-02', '2017-12-16', '2018-03-24', '2018-04-21', '2019-11-16', '2020-03-05', '2020-05-10'], 
                          'age':33, 'gender':'M', 
                          'race_count':2, 
                          'training_elevation':5280, 
                          'max_hr':185}
                    }

In [33]:
races

['2016-04-12',
 '2016-07-17',
 '2016-09-25',
 '2016-09-25',
 '2016-12-03',
 '2016-12-17',
 '2017-01-07',
 '2017-01-28',
 '2017-04-09',
 '2017-04-09',
 '2017-04-22',
 '2017-04-22',
 '2017-04-22',
 '2017-11-11',
 '2017-12-02',
 '2017-12-16',
 '2018-03-24',
 '2018-04-21',
 '2019-11-16',
 '2020-03-05',
 '2020-05-10']

In [39]:
import os

In [40]:
parent_path = './data/races'

In [41]:
athlete_num = 9

In [42]:
directory = f'athlete_{athlete_num}'

In [44]:
path = os.path.join(parent_path, directory)

In [45]:
os.mkdir(path)

In [47]:
import warnings
warnings.filterwarnings("ignore")

In [49]:
set_up_for_prediction(clean_filepath, test_athlete_dict)

A big problem with my data set is correcting pace, for the average elevation gain, and the heart rate of the individual.

Correcting for elevation gain should be as simple as an energy balance: The potential energy taken away by moving the runner in the vertical direction should show a reduction in the speed in the horizontal direction, with some efficiency rate.

Also, heart rate should be a measure of the energy going in to the "system" and so should essentially be a gauge on the throttle of sorts. 100% throttle at a certain grade should yield a certain speed, or pace.

$$ 0.5m\nu_f^2 = 0.5m\nu_i^2 - mgh$$

This simplifies down to:

$$ \nu_f = \sqrt{\nu_i^2-2gdC\rho}$$

Where ν_f is our final velocity, g is the gravitational constant, d is our distance, C is the feet per mile, and ρ is an efficiency constant.

In [ ]:
#lets see if we can correct pace